In [49]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary

#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
# relative phase toffoli gate
from qiskit.circuit.library import RC3XGate, C4XGate, TdgGate

# import basic plot tools
from qiskit.visualization import plot_histogram 


In [2]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')

In [3]:
# no optimization - design 0
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
qc.mcx(control_qubits, target_qubit[0], ancilla_qubits=ancilla_qubits, mode='recursion')
qc.measure(target_qubit[0], classical_bits[0])

In [4]:
qc.draw()

c_0: ──■──────
         │      
  c_1: ──■──────
         │      
  c_2: ──■──────
         │      
  c_3: ──■──────
         │      
  c_4: ──■──────
         │      
  c_5: ──■──────
         │      
  c_6: ──■──────
         │      
  c_7: ──■──────
         │      
  c_8: ──■──────
         │      
  c_9: ──■──────
         │      
 c_10: ──■──────
         │      
 c_11: ──■──────
         │      
 c_12: ──■──────
         │      
 c_13: ──■──────
       ┌─┴──┐┌─┐
    t: ┤0   ├┤M├
       │  X │└╥┘
  a_0: ┤1   ├─╫─
       └────┘ ║ 
  a_1: ───────╫─
              ║ 
  a_2: ───────╫─
              ║ 
  a_3: ───────╫─
              ║ 
  a_4: ───────╫─
              ║ 
out: 1/═══════╩═
              0

In [5]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

890
OrderedDict([('u', 712), ('cx', 656), ('measure', 1)])


In [81]:
def Rc4x():
    # relative phase toffoli-5 implementation
    qc = QuantumCircuit(5)
    qc.h(4)
    qc.t(4)
    qc.cx(3, 4)
    qc.append(TdgGate(), [4])
    qc.mcx([0, 1, 2], 4)
    qc.t(4)
    qc.cx(3, 4)
    qc.append(TdgGate(), [4])
    qc.h(4)

    rc4x = qc.to_gate()
    rc4x.name = "Rc4x"
    return rc4x

qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(13):
    qc.x(control_qubits[i])

for i in range(3):
    qc.append(RC3XGate(), control_qubits[3*i:3*(i+1)]+ancilla_qubits[i:i+1])
# 0 - 8
qc.append(Rc4x(), control_qubits[9:13]+ancilla_qubits[3:4])

qc.append(RC3XGate(), ancilla_qubits[0:2] + control_qubits[13:14] + ancilla_qubits[4:5])

qc.mcx(ancilla_qubits[2:5], target_qubit[0])

# # uncompute
qc.append(RC3XGate().inverse(), ancilla_qubits[0:2] + control_qubits[13:14] + ancilla_qubits[4:5])
for i in range(3):
    qc.append(RC3XGate().inverse(), control_qubits[3*i:3*(i+1)]+ancilla_qubits[i:i+1])
# # 0 - 8
qc.append(Rc4x().inverse(), control_qubits[9:13]+ancilla_qubits[3:4])

qc.measure(target_qubit[0], classical_bits[0])

qc.draw()

┌───┐┌────────┐                                                         »
  c_0: ┤ X ├┤0       ├─────────────────────────────────────────────────────────»
       ├───┤│        │                                                         »
  c_1: ┤ X ├┤1       ├─────────────────────────────────────────────────────────»
       ├───┤│        │                                                         »
  c_2: ┤ X ├┤2       ├─────────────────────────────────────────────────────────»
       ├───┤│        │┌────────┐                                               »
  c_3: ┤ X ├┤        ├┤0       ├───────────────────────────────────────────────»
       ├───┤│        ││        │                                               »
  c_4: ┤ X ├┤        ├┤1       ├───────────────────────────────────────────────»
       ├───┤│        ││        │                                               »
  c_5: ┤ X ├┤        ├┤2       ├───────────────────────────────────────────────»
       ├───┤│        ││        │┌────────┐                                     »
  c_6: ┤ X ├┤        ├┤        ├┤0       ├─────────────────────────────────────»
       ├───┤│        ││        ││        │                                     »
  c_7: ┤ X ├┤        ├┤        ├┤1       ├─────────────────────────────────────»
       ├───┤│  Rcccx ││        ││        │                                     »
  c_8: ┤ X ├┤        ├┤        ├┤2       ├─────────────────────────────────────»
       ├───┤│        ││        ││        │┌───────┐                            »
  c_9: ┤ X ├┤        ├┤        ├┤        ├┤0      ├────────────────────────────»
       ├───┤│        ││  Rcccx ││        ││       │                            »
 c_10: ┤ X ├┤        ├┤        ├┤        ├┤1      ├────────────────────────────»
       ├───┤│        ││        ││        ││       │                            »
 c_11: ┤ X ├┤        ├┤        ├┤        ├┤2      ├────────────────────────────»
       ├───┤│        ││        ││  Rcccx ││       │                            »
 c_12: ┤ X ├┤        ├┤        ├┤        ├┤3      ├────────────────────────────»
       └───┘│        ││        ││        ││       │┌────────┐     ┌───────────┐»
 c_13: ─────┤        ├┤        ├┤        ├┤       ├┤2       ├─────┤2          ├»
            │        ││        ││        ││  Rc4x ││        │┌───┐│           │»
    t: ─────┤        ├┤        ├┤        ├┤       ├┤        ├┤ X ├┤           ├»
            │        ││        ││        ││       ││        │└─┬─┘│           │»
  a_0: ─────┤3       ├┤        ├┤        ├┤       ├┤0       ├──┼──┤0          ├»
            └────────┘│        ││        ││       ││        │  │  │           │»
  a_1: ───────────────┤3       ├┤        ├┤       ├┤1 Rcccx ├──┼──┤1 rcccx_dg ├»
                      └────────┘│        ││       ││        │  │  │           │»
  a_2: ─────────────────────────┤3       ├┤       ├┤        ├──■──┤           ├»
                                └────────┘│       ││        │  │  │           │»
  a_3: ───────────────────────────────────┤4      ├┤        ├──■──┤           ├»
                                          └───────┘│        │  │  │           │»
  a_4: ────────────────────────────────────────────┤3       ├──■──┤3          ├»
                                                   └────────┘     └───────────┘»
out: 1/════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                   ┌───────────┐             
«  c_0: ────────────────────────────┤0          ├─────────────
«                                   │           │             
«  c_1: ────────────────────────────┤1          ├─────────────
«                                   │           │             
«  c_2: ────────────────────────────┤2          ├─────────────
«                                   │           │┌───────────┐
«  c_3: ────────────────────────────┤           ├┤0          ├
«                   

In [82]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

75
OrderedDict([('u', 115), ('cx', 94), ('measure', 1)])


In [83]:
qobj = assemble(qc_transpiled)
result = aer_simulator.run(qobj).result()
print(result.get_counts())

{'0': 1024}


In [39]:
# test efficiency of various sizes with no-ancilla
numControls = 10
modes = ['noancilla', 'recursion', 'v-chain', 'v-chain-dirty']
for m in modes:
    print(f"Mode: {m}")
    for i in range(1, numControls):
        ancilla_qubits = QuantumRegister(15, name='a')
        control_qubits = QuantumRegister(i, name="c")
        qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
        for j in range(i):
            qc.x(control_qubits[j])
        qc.mcx(control_qubits, ancilla_qubits[0], ancilla_qubits=ancilla_qubits[1:], mode=m)
        qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
        if (m == 'noancilla'):
            num_ancilla = 0
        elif (m == 'recursion'):
            num_ancilla = 0 if i < 5 else 1
        elif (m == 'v-chain' or m =='v-chain-dirty'):
            num_ancilla = i - 2 if i > 2 else 0
        print(f"Controls: {i}, Depth: {qc_transpiled.depth()}, Ancilla: {num_ancilla}, Ops: {qc_transpiled.count_ops()}")
    print()

Mode: noancilla
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65, Ancilla: 0, Ops: OrderedDict([('u', 44), ('cx', 36)])
Controls: 5, Depth: 155, Ancilla: 0, Ops: OrderedDict([('u', 98), ('cx', 92)])
Controls: 6, Depth: 315, Ancilla: 0, Ops: OrderedDict([('u', 195), ('cx', 188)])
Controls: 7, Depth: 635, Ancilla: 0, Ops: OrderedDict([('u', 388), ('cx', 380)])
Controls: 8, Depth: 1275, Ancilla: 0, Ops: OrderedDict([('u', 773), ('cx', 764)])
Controls: 9, Depth: 2555, Ancilla: 0, Ops: OrderedDict([('u', 1542), ('cx', 1532)])

Mode: recursion
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65

In [70]:
# Testing my gate
qc = QuantumCircuit(5)
qc.h(4)
qc.t(4)
qc.cx(3, 4)
qc.append(TdgGate(), [4])
qc.mcx([0, 1, 2], 4)
qc.t(4)
qc.cx(3, 4)
qc.append(TdgGate(), [4])
qc.h(4)
#circuit already defined
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
matrix = result.get_unitary(qc)
qc.draw()

q_0: ────────────────────────■────────────────────────
                             │                        
q_1: ────────────────────────■────────────────────────
                             │                        
q_2: ────────────────────────■────────────────────────
                             │                        
q_3: ────────────■───────────┼─────────■──────────────
     ┌───┐┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌─────┐┌───┐
q_4: ┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ H ├
     └───┘└───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘

In [71]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

29
OrderedDict([('u', 18), ('cx', 16)])


In [84]:
matrix[8, :] 
# |01111> -> -i|11111>
# |11111> -> i|01111>
# |00111> -> 

<ipython-input-84-b0ef0dd27018>:1: DeprecationWarning: The return type of saved unitaries has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Operator` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  matrix[8, :]


array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        1.00000000e+00-5.55111512e-17j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
       -1.11022302e-16+5.55111512e-17j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000